# <center> An approach to predicting car sensor failure </center> 

In [1]:
# reading packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import random
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_auc_score
import xgboost
from category_encoders.one_hot import OneHotEncoder


# 1. Reading data 

In [2]:
# reading files with the help of a generator -  faster than using for loop
sensor_data = glob.glob("*.csv")     
df_from_each_file = (pd.read_csv(f) for f in sensor_data)
df = pd.concat(df_from_each_file, axis = 0, ignore_index=True)

# sort datafame by date for groupby and feature engineering purposes
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df = df.sort_values(by='date')


In [3]:
# number of rows and columns
df.shape

(10982984, 95)

In [3]:
# print value counts of sensors which had failures
failures = df[df['failure'] == 1]
serial_numbers = list(failures['serial_number'].unique())
subset_of_failures = df[df['serial_number'].isin(serial_numbers)]
subset_of_failures['serial_number'].value_counts()


Z300PMDR           175
MK0311YHGHYWMA     175
W300AZSC           175
Z303ZA70           174
JK1105B8GG4X7X     173
                  ... 
W3006E5B             1
WD-WCAU45402975      1
Z300KHN0             1
WD-WCC4N5AHPH31      1
Z3015V4D             1
Name: serial_number, Length: 697, dtype: int64

In [5]:
# number of unique days
print(len(df['date'].unique()))

175


In [6]:
print(df.columns)

Index(['date', 'serial_number', 'type', 'capacity', 'failure',
       'stat_1_normalized', 'stat_1_raw', 'stat_2_normalized', 'stat_2_raw',
       'stat_3_normalized', 'stat_3_raw', 'stat_4_normalized', 'stat_4_raw',
       'stat_5_normalized', 'stat_5_raw', 'stat_7_normalized', 'stat_7_raw',
       'stat_8_normalized', 'stat_8_raw', 'stat_9_normalized', 'stat_9_raw',
       'stat_10_normalized', 'stat_10_raw', 'stat_11_normalized',
       'stat_11_raw', 'stat_12_normalized', 'stat_12_raw',
       'stat_13_normalized', 'stat_13_raw', 'stat_15_normalized',
       'stat_15_raw', 'stat_22_normalized', 'stat_22_raw',
       'stat_183_normalized', 'stat_183_raw', 'stat_184_normalized',
       'stat_184_raw', 'stat_187_normalized', 'stat_187_raw',
       'stat_188_normalized', 'stat_188_raw', 'stat_189_normalized',
       'stat_189_raw', 'stat_190_normalized', 'stat_190_raw',
       'stat_191_normalized', 'stat_191_raw', 'stat_192_normalized',
       'stat_192_raw', 'stat_193_normalized', 's

In [7]:
# ratio of positive and negative values in the target variable
df['failure'].value_counts()


0    10982287
1         697
Name: failure, dtype: int64

In [8]:
print("dataset is heavily imbalanced")

dataset is heavily imbalanced


In [9]:
# total number of sensors tracked
len(df['serial_number'].unique())

70923

In [10]:
# total number of sensor types
len(df['type'].unique())

68

In [22]:
# average monthly failure rate 
df = df.set_index('date')
monthly_average_of_failures = df.failure.resample('M').sum()[:5].mean()
monthly_average_of_nunique = df.serial_number.resample('M').nunique()[:5].mean()
average_failure_rate = (monthly_average_of_failures/monthly_average_of_nunique)*100
df.reset_index(inplace=True) 
print("average monthly failure rate is {}".format(average_failure_rate))

average monthly failure rate is 0.1916313606142306


In [33]:
# average number of sensor failures in the following 7 days
n_failures_in_following_7d = df.groupby('date')['failure'].sum()
n_failures_in_following_7d = n_failures_in_following_7d.sort_index(ascending = False )
n_failures_in_following_7d = n_failures_in_following_7d.rolling(window = 7, min_periods = 7).sum()
n_failures_in_following_7d = n_failures_in_following_7d.sort_index(ascending = True ).mean()
print("Average number of failures in the following 7 days, for each day in the dataset, is {:.2f}.".format(n_failures_in_following_7d))
print("In other words, model would need to predict failure in the following 7 days for, in average, {:.2f} out of {} number of monthly active sensors.".\
      format(n_failures_in_following_7d, monthly_average_of_nunique))
print("Tough job!")

Average number of failures in the following 7 days, for each day in the dataset, is 27.94.
In other words, model would need to predict failure in the following 7 days for, in average, 27.94 out of 63350.8 number of monthly active sensors.
Tough job!


In [14]:
n_failures_in_following_7d

date
2020-06-23    4
2020-06-22    2
2020-06-21    4
2020-06-20    0
2020-06-19    5
             ..
2020-01-05    4
2020-01-04    3
2020-01-03    4
2020-01-02    6
2020-01-01    4
Name: failure, Length: 175, dtype: int64

In [5]:
n_failures_in_following_7d

date
2020-01-01    4
2020-01-02    6
2020-01-03    4
2020-01-04    3
2020-01-05    4
             ..
2020-06-19    5
2020-06-20    0
2020-06-21    4
2020-06-22    2
2020-06-23    4
Name: failure, Length: 175, dtype: int64

# 2. Data wrangling and feature engineering

In [34]:
# defining variables for future use

target_variable = 'failure'

numerical_features = ['stat_1_normalized', 'stat_1_raw', 'stat_2_normalized', 'stat_2_raw', 'stat_3_normalized', 'stat_3_raw', 'stat_4_normalized', 'stat_4_raw', 'stat_5_normalized', 'stat_5_raw', 'stat_7_normalized', 'stat_7_raw', 'stat_8_normalized', 'stat_8_raw', 'stat_9_normalized', 'stat_9_raw', 'stat_10_normalized', 'stat_10_raw', 'stat_11_normalized', 'stat_11_raw', 'stat_12_normalized', 'stat_12_raw', 'stat_13_normalized', 'stat_13_raw', 'stat_15_normalized', 'stat_15_raw', 'stat_22_normalized', 'stat_22_raw', 'stat_183_normalized', 'stat_183_raw', 'stat_184_normalized', 'stat_184_raw', 'stat_187_normalized', 'stat_187_raw', 'stat_188_normalized', 'stat_188_raw', 'stat_189_normalized', 'stat_189_raw', 'stat_190_normalized', 'stat_190_raw', 'stat_191_normalized', 'stat_191_raw', 'stat_192_normalized', 'stat_192_raw', 'stat_193_normalized', 'stat_193_raw', 'stat_194_normalized', 'stat_194_raw', 'stat_195_normalized', 'stat_195_raw', 'stat_196_normalized', 'stat_196_raw', 'stat_197_normalized', 'stat_197_raw', 'stat_198_normalized', 'stat_198_raw', 'stat_199_normalized', 'stat_199_raw', 'stat_200_normalized', 'stat_200_raw', 'stat_201_normalized', 'stat_201_raw', 'stat_220_normalized', 'stat_220_raw', 'stat_222_normalized', 'stat_222_raw', 'stat_223_normalized', 'stat_223_raw', 'stat_224_normalized', 'stat_224_raw', 'stat_225_normalized', 'stat_225_raw', 'stat_226_normalized', 'stat_226_raw', 'stat_240_normalized', 'stat_240_raw', 'stat_241_normalized', 'stat_241_raw', 'stat_242_normalized', 'stat_242_raw', 'stat_250_normalized', 'stat_250_raw', 'stat_251_normalized', 'stat_251_raw', 'stat_252_normalized', 'stat_252_raw', 'stat_254_normalized', 'stat_254_raw', 'stat_255_normalized', 'stat_255_raw' ]

numerical_features_normalized = [x for x in numerical_features if 'raw' not in x]

categorical_features = ['type']

best_indicators = ['stat_5_normalized','stat_10_normalized','stat_184_normalized','stat_187_normalized','stat_188_normalized','stat_196_normalized','stat_197_normalized','stat_198_normalized','stat_201_normalized']

print("best indicators were marked as best in the data science assignment pdf")

best indicators were marked as best in the data science assignment pdf


In [35]:
# subsetting
df = df[['date', 'serial_number', 'capacity'] + best_indicators + categorical_features + [target_variable]]
# df = df[['date', 'serial_number', 'capacity'] + numerical_features_normalized + categorical_features + [target_variable]]

In [36]:
# randomly dropping some sensors which didnt have failure - will solve memory issues due to dataset size/computer capacity, and should not affect the modelling too much because the dataset is heavily imbalanced as is
print("Dataset shape before semi random subset:", df.shape)
serial_n_to_keep = df['serial_number'].unique()
serial_n_to_keep = [x for x in serial_n_to_keep if x not in serial_numbers]
serial_n_to_keep = random.sample(serial_n_to_keep, int(0.05*len(serial_n_to_keep)))
serial_n_to_keep = serial_n_to_keep + serial_numbers
df = df[df['serial_number'].isin(serial_n_to_keep)]
print("Dataset shape before after semi random subset:", df.shape)




Dataset shape before semi random subset: (10982984, 14)
Dataset shape before after semi random subset: (605636, 14)


In [8]:
# count number of nan values in columns
for col in df.columns:
    na_sum = df[col].isna().sum()
    if na_sum>0:
        print(col, na_sum)

stat_5_normalized 2
stat_10_normalized 2
stat_184_normalized 255337
stat_187_normalized 255337
stat_188_normalized 255337
stat_196_normalized 356188
stat_197_normalized 2
stat_198_normalized 2
stat_201_normalized 611523


In [37]:
# creating features which will collect the stat information from past X days for each day and for each serial_number
n_days_shift = 14

for i in range(1,n_days_shift +1):
    best_indicators_shifted = ["{}_shift{}".format(x,i) for x in best_indicators]
    df[best_indicators_shifted] = df.groupby(['serial_number'])[best_indicators].shift(i)
#     numerical_features_mormalized_shifted = ["{}_shift{}".format(x,i) for x in numerical_features_normalized]
#     df[numerical_features_mormalized_shifted] = df.groupby(['serial_number'])[numerical_features_normalized].shift(i)
    print("creating variables with timeshift {}, new dataframe shape is {}".format(i,df.shape))


creating variables with timeshift 1, new dataframe shape is (605636, 23)
creating variables with timeshift 2, new dataframe shape is (605636, 32)
creating variables with timeshift 3, new dataframe shape is (605636, 41)
creating variables with timeshift 4, new dataframe shape is (605636, 50)
creating variables with timeshift 5, new dataframe shape is (605636, 59)
creating variables with timeshift 6, new dataframe shape is (605636, 68)
creating variables with timeshift 7, new dataframe shape is (605636, 77)
creating variables with timeshift 8, new dataframe shape is (605636, 86)
creating variables with timeshift 9, new dataframe shape is (605636, 95)
creating variables with timeshift 10, new dataframe shape is (605636, 104)
creating variables with timeshift 11, new dataframe shape is (605636, 113)
creating variables with timeshift 12, new dataframe shape is (605636, 122)
creating variables with timeshift 13, new dataframe shape is (605636, 131)
creating variables with timeshift 14, new d

In [38]:
# reverse order because rolling window calculates backwards
df = df.sort_values(by='date', ascending = False )

# create target variable
df['Failure_in_following_7d'] = df.groupby('serial_number')['failure'].rolling(window = 7, min_periods = 1).sum().reset_index(0,drop=True)

# reverse back to normal
df = df.sort_values(by='date', ascending = True )


# 3. Walk forward time series modelling

### 3.1 preparing hyperparameter  dictionary for gridsearch purposes

In [40]:
# preparing hyperparameter grid for xgboost algorithm
xgb_param_grid ={
    'nthread': [1],  # use maximum number of threads
    'objective': ['binary:logistic'],
    "grow_policy": ["lossguide"],
    "n_jobs": [-1],
    "num_class": [1],
    "tree_method": ["hist"],
    "max_depth": [5, 6, 13], 
#     "min_child_weight": [2,5],  
#     "colsample_bytree": [0.7,0.85],
#     "gamma": [0, 1],
    "subsample": [0.7,0.9],
    'learning_rate': [0.01,  0.1],
    'silent': [1],
    'seed': [1337],
    "n_estimators": [8,16,24]
    }



### 3.2 defining starting period for walk forward method

In [41]:
# train validation test splits
sorted_list_of_dates= list(df.date.unique())
sorted_list_of_dates.sort()


# # initial_dates = [x for x in sorted_list_of_datesif x <= np.datetime64("2009-06-30T00:00:00")]
# # iteration_dates = [x for x in sorted_list_of_datesif x> np.datetime64("2009-06-30T00:00:00")]

# initial dates will be used for train data from the beggining 
initial_dates = [x for x in sorted_list_of_dates if x <= np.datetime64("2020-05-22T00:00:00")]

# iteration dates will be iterated over and used as both as train and test sets
iteration_dates = [x for x in sorted_list_of_dates if x> np.datetime64("2020-05-22T00:00:00")]

# creating an additional helper list which is a shifted version of the iteration dates
shifted_dates = iteration_dates.copy()
shifted_dates.pop(0)
# inserting a dummy value to keep the length of the lists the same
shifted_dates.append('dummy')


In [42]:
# perform one hot encoding
encoder_obj = OneHotEncoder(cols=categorical_features,
                                    use_cat_names=True,
                                    handle_missing='return_nan',  # 'error' option has a bug?
                                    handle_unknown='return_nan')
encoder_obj.fit(df)
print(df.shape)
df = encoder_obj.transform(df)
print(df.shape)

(605636, 141)
(605636, 187)


In [43]:
non_predictor_cols = ['date', 'serial_number','Failure_in_following_7d']
categorical_columns = [x for x in df.columns if 'type' in x]
numerical_columns = [x for x in df.columns if x not in non_predictor_cols]
numerical_columns = [x for x in numerical_columns if x not in categorical_columns]
predictor_features = numerical_columns + categorical_columns

### 3.3 implementing the walk forward method

In [51]:
counter = 1
auc_list = []
predicted_actuals = []
loop_length = len(iteration_dates)

# starting the iteration
for i,j in zip(iteration_dates, shifted_dates):

    # first element of iteration date is also included in the train set
    initial_dates.append(i)

    
    # defining train set
    train_set = df[df['date'].isin(initial_dates)]
    X_train = train_set[train_set.columns[~train_set.columns.isin(['Failure_in_following_7d'])]]
    y_train = train_set['Failure_in_following_7d']

    # apply final model on latest data if iteration is not at end of the list (prediction point)
    if i == iteration_dates[-7]:
        print("\n\n\n\nFinal prediction date is: {}".format(i))
        
        final_prediction_data = df[df['date']==i]       
        prob_score = model.predict_proba(final_prediction_data[predictor_features])[:,1]
        break
        
    # otherwise, perform walk forward method
    else:
        print('\n\nCurrent latest train date', initial_dates[-1])
        print("Current test and/or validation date", j)
        
        # defining test_validation_set - for hyperparametrization purposes half of the set will be used for validation half for testing
        test_validation_set =df[df['date']==j]
        
        # defining x and y
        X = test_validation_set[test_validation_set.columns[~test_validation_set.columns.isin(['Failure_in_following_7d'])]]
        y = test_validation_set['Failure_in_following_7d']
        
        # perform gridsearch again at three different time points, since the training data increases and perhaps optimal hyperparameters change
        if counter in [1,9,25]: 

            # for hyperparametrization we need to have both train and test sdata
            X_validation, X_test, y_validation, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.6)

            # performing hyperparametrization
            best_score = 0
            best_params = None
            len_params = len(ParameterGrid(xgb_param_grid))
                
            for param,param_counter in zip(ParameterGrid(xgb_param_grid), range(1, len_params+1)):
               
                # fit the model
                model = xgboost.XGBClassifier(**param)
                model.fit(X_train[predictor_features],y_train)

                # get auc - a common classification metric
                predicted_values = model.predict_proba(X_validation[predictor_features])[:,1]
                auc = roc_auc_score(y_validation, predicted_values)

                # save results of the best score
                if auc > best_score:
                    best_score = auc
                    best_params = model.get_xgb_params()
                    
                    if param_counter == len_params:
                        predicted_actuals.append([predicted_values, y_validation])

            auc_list.append(best_score)
            print('best_score in hyperparametrization in step {}'.format(counter), best_score)
            print('best_params in hyperparametrization in step {}'.format(counter), best_params)
        
        # otherwise,  use the current date as the test date before adding it to the training in the next loop iteration 
        else:
            model = xgboost.XGBClassifier(**best_params)
            model.fit(X_train[predictor_features],y_train)
            
            # if iteration is at the end of the list (latest date), do not use it as a test set
            if counter == loop_length -7:
                print("Latest date -7 is the latest date used for training. Model will be trained, and latest date will be for prediction")
            
            # otherwise, use it as a test set
            else:
                predicted_values = model.predict_proba(X[predictor_features])[:,1]
                auc = roc_auc_score(y, predicted_values)
                auc_list.append(auc)
                predicted_actuals.append([predicted_values, y])

       # changing parameters for next loop iteration
        counter+= 1
        print("auc_list", auc_list)



Current latest train date 2020-05-23T00:00:00.000000000
Current test and/or validation date 2020-05-24T00:00:00.000000000
best_score in hyperparametrization in step 1 0.8109813573523251
best_params in hyperparametrization in step 1 {'objective': 'binary:logistic', 'base_score': 0.5, 'booster': None, 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'gamma': 0, 'gpu_id': -1, 'importance_type': 'gain', 'interaction_constraints': None, 'learning_rate': 0.1, 'max_delta_step': 0, 'max_depth': 13, 'min_child_weight': 1, 'missing': nan, 'monotone_constraints': None, 'n_estimators': 40, 'n_jobs': -1, 'num_parallel_tree': 1, 'random_state': 1337, 'reg_alpha': 0, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 1, 'tree_method': 'hist', 'validate_parameters': False, 'verbosity': None, 'grow_policy': 'lossguide', 'nthread': 1, 'num_class': 1, 'seed': 1337, 'silent': 1}
auc_list [0.8109813573523251]


Current latest train date 2020-05-24T00:00:00.000000000
Current test an



Current latest train date 2020-06-12T00:00:00.000000000
Current test and/or validation date 2020-06-13T00:00:00.000000000
auc_list [0.8109813573523251, 0.8175573360776243, 0.8506707237272608, 0.8546902302394904, 0.8517141889357995, 0.8487804154302669, 0.8502090425167349, 0.8418670678735227, 0.8006811877957253, 0.8233986603072704, 0.7930932402360099, 0.7721229278460866, 0.7751091299200947, 0.8071766895035102, 0.9235864416814684, 0.8597573490843575, 0.837850812407681, 0.8734898289333287, 0.8334286133098798, 0.8170932065289985, 0.8001143206411011]


Current latest train date 2020-06-13T00:00:00.000000000
Current test and/or validation date 2020-06-14T00:00:00.000000000
auc_list [0.8109813573523251, 0.8175573360776243, 0.8506707237272608, 0.8546902302394904, 0.8517141889357995, 0.8487804154302669, 0.8502090425167349, 0.8418670678735227, 0.8006811877957253, 0.8233986603072704, 0.7930932402360099, 0.7721229278460866, 0.7751091299200947, 0.8071766895035102, 0.9235864416814684, 0.85975734908

In [54]:
print("AUC across test sets", auc_list)

AUC across test sets [0.8109813573523251, 0.8175573360776243, 0.8506707237272608, 0.8546902302394904, 0.8517141889357995, 0.8487804154302669, 0.8502090425167349, 0.8418670678735227, 0.8006811877957253, 0.8233986603072704, 0.7930932402360099, 0.7721229278460866, 0.7751091299200947, 0.8071766895035102, 0.9235864416814684, 0.8597573490843575, 0.837850812407681, 0.8734898289333287, 0.8334286133098798, 0.8170932065289985, 0.8001143206411011, 0.8162860506180105, 0.7525876867105634, 0.7151984946145961, 0.7311224904494839]


In [57]:
print("Average auc", np.mean(auc_list))

Average auc 0.8183426997096477


# 4. Explanation and reasoning

#### 4.1 Disclaimer 

- Dataset is relatively large and problematic for most personal computers such as mine
- Analysis was performed on Amazon virtual instance with 32 GB RAM and 12 cores
- Even in this configuration, only a subset of the dataset was used for training adue to RAM constraint and task time limit
- Dataset has an intrinsic time component which means the train and test sets need to be carefully designed. No crossvalidation or any type of traditional random splits would be okay in this situation
- Because of this, I implemented a so called walk forward method of training a model which allowed me to incrementally increase training set size and ammount of test results
- extreme gradient boosting, a state of the art Machine Learning algorithm, was used to create a predictive model
- hyperparametrization was performed in multiple steps over a time period
- dataset is extremely balanced which makes problem solution very challenging
- Amazon instance had problems with the matplot library and for this reason no plots are present in this notebook
- only basic feature engineering was done due to time constraints for the task

#### 4.2 Results

- AUC was used as an evaluation metric, since accuracy is pointles for imbalanced datasets. 
- Average AUC across 25 test sets was 0.81. Perfect model has AUC 1, random model would have AUC 0.5
- whether 0.81 or not is a good score, would depend on what the baseline in the industry for these types of models is, and this is not something that I am currently familiar with
- For example, in psychological studies AUC higher than 0,7 can in some cases be considered very good, AUC of 0,8 of a model identifying profitable investments in banking could be excellent, while AUC of 0,95  when classifying handwritten digits could be nothing special at all. 
- it is fair to mention that average AUC of 0.81 is probably too optimistic for two or more reasons:
    - AUC across test sets is not stable enough to make any final conclusions
    - large ammount of sensors with no failures were dropped due to memory constraints. While this may not always significantly affect the quality of model, there is a chance that this artificial balancing of the dataset made the evaluation overly optimistic

#### 4.3 Conclusion and recommendations

- A typical Machine Learning approach was used to demonstrate the effectivenes in solving car sensors failure prediction
- Results might be decent at first glance, though somewhat unstable and overly optimistic to certain degree
- If this was a real project my recommendation would be:
    - first of all, perform preliminary exploratory analysis for interpretation purposes. Idealy, implement a method called Weight of Evidence, which will analyse the effects on the target varible for all features in univariate fashion
    - check if any features are very obvious indicators of failure. In some cases model is not needed at all, a good rule based method might be more appropriate

    - further explore the walk forward method but:
        - use stronger machine, perhaps even a cluster if data is even bigger in reality
        - put strong focus on feature engineering
        - perform more detailed hypetparametrization
    - Explore additional approaches towards solving the problem:
        - some other ML algorithms
        - anomaly detection methods
        - stacking models, e.g. an ML model and a rule based event model on top 
        - although I am not a fan of dataset balancing in scenarios where data is naturally imbalanced such as here, trying
    SMOTE method or something similar might be worth a try
    - AUC, although one of the best evaluation metrics across industries, might not be the optimal in a heavily imbalanced scenario. Lift curve might be worth considering if the bussines goal would bo to replace in advance certain number of sensors for which the model gave highest probability of failure. Also, precision-recall, or any similar decision threshold dependant metric could be useful if bussiness can decide on the optimal ratio of true positives vs. false positives 